# Statsmodels

In [5]:
import statsmodels.api as sm # Uses numpy arrays
import statsmodels.formula.api as smf # Uses pandas
from statsmodels.stats.outliers_influence import OLSInfluence

In [ ]:
####### Example with pandas

## Fit regression model (using the natural log of one of the regressors)
model = smf.ols(formula, data)
results = model.fit()

## Inspect the results
print(results.summary())

## Some results
results.params
results.pvalues
results.model.endog_names # Dependent variables
results.rsquared
results.resid
results.fittedvalues
results.resid_pearson    # Normalised residuals
OLSInfluence(results).influence  # Leverage

## Check classification encoding
np.column_stack((df[Response].values, results.model.endog))

## Convert classification predictions (from model output) to response labels
probablity_threshold = 0.5
predictions_nominal = [ "Up" if x < probablity_threshold else "Down" for x in predictions]

## Prediction
# you have to create a DataFrame since the Statsmodels formula interface expects it
# if no args passed to .predict(), model will take original training data to predict
X_new = pd.DataFrame({'TV': [50]})
X_new.head()
predictions = lmres.predict(X_new)

## Prediction with confidence intervals
new = pd.DataFrame([[1, 5], [1, 10], [1, 15]], columns=['Intercept', 'lstat'])

def predict_conf_int(res, new):

    # Get the predicted values
    fit = pd.DataFrame(res.predict(new), columns=['fit'])
    
    # Get the confidence interval for the model (and rename the columns to something a bit more useful)
    ci = res.conf_int().rename(columns={0: 'lower', 1: 'upper'})
    
    # Now a little bit of matrix multiplication to get the confidence intervals for the predictions
    ci = ci.T.dot(new.T).T
    
    # And finally wrap up the confidence intervals with the predicted values
    return pd.concat([fit, ci], axis=1)

predict_conf_int(lmres, new)

## Residual plots
fitted_values = pd.Series(result.fittedvalues, name="Fitted Values")
residuals = pd.Series(result.resid, name="Residuals")
sns.regplot(fitted_values, residuals, fit_reg=False)

## Leverage plot
from statsmodels.stats.outliers_influence import OLSInfluence
leverage = pd.Series(OLSInfluence(results).influence, name = "Leverage")
sns.regplot(leverage, s_residuals,  fit_reg=False)

## ANOVA between two models
sm.stats.anova_lm(lm.fit, lm.fit2)

## Example with Numpy

import numpy as np

import statsmodels.api as sm

#### Generate artificial data (2 regressors + constant)
nobs = 100

X = np.random.random((nobs, 2))

X = sm.add_constant(X)

beta = [1, .1, .5]

e = np.random.random(nobs)

y = np.dot(X, beta) + e

#### Fit regression model
results = sm.OLS(y, X).fit()

#### Inspect the results
print(results.summary())


## Model Selection

Subset Selection

In [ ]:
##### Best Subset Selection
import itertools 

def processSubset(feature_set):
    """Fit model on feature_set and calculate RSS""" 
    model = sm.OLS(y,X[list(feature_set)])
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

def getBest(k):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

models = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
for i in range(1,8):
    models.loc[i] = getBest(i)

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

models # DF with best models for each k paramater
print(models.loc[2, "model"].summary()) # model summary

In [ ]:
##### Forward Selection

def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

models2 = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models2.loc[i] = forward(predictors)
    predictors = models2.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

In [ ]:
###### Backward Selection


def backward(predictors):
    
    tic = time.time()
    
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors)-1):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the lowest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)-1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

models3 = pd.DataFrame(columns=["RSS", "model"], index = range(1,len(X.columns)))

tic = time.time()
predictors = X.columns

while(len(predictors) > 1):  
    models3.loc[len(predictors)-1] = backward(predictors)
    predictors = models3.loc[len(predictors)-1]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

## R Syntax Formula

##### Linear Regresssion
medv ~ lstat              

##### Multiple Linear Regression
medv ~ lstat + age

'medv ~ ' + '+'.join(df.columns.difference(['medv', 'age']) ---> use all other variables as predictors except the ones in list

##### Interaction Terms

medv ~ lstat*age or lstat:age

##### Non-linear transform

medv ~ lstat + np.square(lstat)

medv ~ np.log(rm)

##### Dummy variables generated automatically

## Models

##### Linear Regression

smf.ols(formula, data)

##### Logistic Regression

smf.glm(formula, data, family = sm.families.Binomial())